In [107]:
import pandas as pd
import numpy as np
import glob

In [108]:
files = glob.glob("csv/*.csv")

In [117]:
mat = np.zeros([31,30])
mat[:,:] = np.nan
mat = pd.DataFrame(data=mat,
                   columns=["subj","age","sex","group","all_RT","all_SD","num_RT","num_SD","num_FA","num_MISS","num2_RT","num2_SD","num2_FA","num2_MISS","fig_RT","fig_SD","fig_FA","fig_MISS","fig2_RT","fig2_SD","fig2_FA","fig2_MISS","cha_RT","cha_SD","cha_FA","cha_MISS","cha2_RT","cha2_SD","cha2_FA","cha2_MISS"])

In [129]:
for go_pretest in range(len(files)):
    # データファイルを選択
    csv_file_name=files[go_pretest]
    csv_file=pd.read_csv(csv_file_name,encoding="Shift-JIS")
    csv_file.iloc[:,2]=csv_file.iloc[:,2]*1000
    
    #各ブロックの開始行を取得
    chara= csv_file[csv_file.values=="文字"].index[0]
    chara2= csv_file[csv_file.values=="文字2"].index[0]
    num= csv_file[csv_file.values=="数字"].index[0]
    num2= csv_file[csv_file.values=="数字2"].index[0]
    fig= csv_file[csv_file.values=="図形"].index[0]
    fig2= csv_file[csv_file.values=="図形2"].index[0]
    
    # グループによってトライアルの順番が異なる
    if csv_file.columns[3]=='A':# A: fig-> num-> chara
        data=[csv_file[num+1:num2],csv_file[num2+1:chara],csv_file[fig+1:fig2],csv_file[fig2+1:num],csv_file[chara+1:chara2],csv_file[chara2+1:len(csv_file)]]
    elif csv_file.columns[3]=="B":# B: chara-> num-> fig
        data=[csv_file[num+1:num2],csv_file[num2+1:fig],csv_file[fig+1:fig2],csv_file[fig2+1:len(csv_file)],csv_file[chara+1:chara2],csv_file[chara2+1:num]]
    else:# C: num-> fig-> chara
        data=[csv_file[num+1:num2],csv_file[num2+1:fig],csv_file[fig+1:fig2],csv_file[fig2+1:chara],csv_file[chara+1:chara2],csv_file[chara2+1:len(csv_file)]]

    mat.iloc[go_pretest,0]=csv_file.columns[2]
    mat.iloc[go_pretest,1]=int(csv_file.columns[0])
    mat.iloc[go_pretest,2]=csv_file.columns[1]
    mat.iloc[go_pretest,3]=csv_file.columns[3]
    mat.iloc[go_pretest,4]=csv_file.iloc[:,2].mean(skipna=True)
    mat.iloc[go_pretest,5]=csv_file.iloc[:,2].std(skipna=True)

    for data_file in range(6):
        dat=data[data_file]

        # 反応時間平均
        mat.iloc[go_pretest,4*data_file+6]=dat.iloc[:,2].mean(skipna=True)
        mat.iloc[go_pretest,4*data_file+7]=dat.iloc[:,2].std(skipna=True)


        # False Alarm Rate
        trial=0
        FA_trial=0
        for FA_trial_num in range(len(dat)):
            if dat.iloc[FA_trial_num,0]=="0":
                trial+=1
            if dat.iloc[FA_trial_num,0]=="0" and dat.iloc[FA_trial_num,1]==1:
                FA_trial+=1
        mat.iloc[go_pretest,4*data_file+8]=FA_trial/trial*100

        # Miss Rate
        trial=0
        Miss_trial=0
        for Miss_trial_num in range(len(dat)):
            if dat.iloc[Miss_trial_num,0]=="1":
                trial+=1
            if dat.iloc[Miss_trial_num,0]=="1" and dat.iloc[Miss_trial_num,1]==0:
                Miss_trial+=1
        mat.iloc[go_pretest,4*data_file+9]=Miss_trial/trial*100
    

In [125]:
mat.to_excel('./analysis.xlsx')

In [128]:
print(mat.all_RT.mean())
print(mat.all_RT.std())

402.224628400944
28.006069793044453
